In [1]:
import boto3
import torch
import sagemaker
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
import boto3
import datetime
import time
from time import strftime,gmtime
import json
import os
import urllib
import sys
import io

boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_client = boto_session.client("sagemaker")
sm_runtime = boto_session.client("sagemaker-runtime")
sns_client = boto3.client('sns')
region = boto_session.region_name
bucket = sm_session.default_bucket()
prefix = 'monai-async-inference-brain-tumor'
role = "arn:aws:iam::617011600974:role/service-role/AmazonSageMaker-ExecutionRole-20210316T155032"

print(region)
print(role)
print(bucket)
print(prefix)









eu-west-1
arn:aws:iam::617011600974:role/service-role/AmazonSageMaker-ExecutionRole-20210316T155032
sagemaker-eu-west-1-617011600974
monai-async-inference-brain-tumor


In [7]:
#model_name = "MONAI-DEMO-MultiModelModel" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
#model_url = "s3://sagemaker-eu-west-1-617011600974/monai-on-sagemaker-docker-2022-06-03-08-24-03-985/output/model.tar.gz"

!mkdir model_and_code
!cp ./model/model.pth model_and_code/
#torch.save(model, 'model_and_code/model.pth')
!mkdir model_and_code/code
!cp ./source/* model_and_code/code
!tar cvzf model.tar.gz -C model_and_code/ . 

from sagemaker.s3 import S3Uploader
file_key = 'model.tar.gz'
model_artifact = S3Uploader.upload(file_key,'s3://{}/{}/model'.format(bucket, prefix))
print(model_artifact)




mkdir: cannot create directory ‘model_and_code’: File exists
mkdir: cannot create directory ‘model_and_code/code’: File exists
./
./model.pth
./code/
./code/inference.py
./code/requirements.txt
s3://sagemaker-eu-west-1-617011600974/monai-async-inference-brain-tumor/model/model.tar.gz


In [8]:
from sagemaker.image_uris import retrieve

deploy_instance_type = 'ml.g4dn.xlarge'
pytorch_inference_image_uri = retrieve('pytorch',
                                       region,
                                       version='1.11.0',
                                       py_version='py38',
                                       instance_type = deploy_instance_type,
                                       accelerator_type=None,
                                       image_scope='inference')
print(pytorch_inference_image_uri)



763104351884.dkr.ecr.eu-west-1.amazonaws.com/pytorch-inference:1.11.0-gpu-py38


In [10]:


container = pytorch_inference_image_uri
model_name = 'sagemaker-monai-brain-tumor-{0}'.format(str(int(time.time())))
print(container)
print(model_name)

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': model_artifact,
        'Environment': {
            'TS_MAX_REQUEST_SIZE': '100000000', #default max request size is 6 Mb for torchserve, need to update it to support the 70 mb input payload
            'TS_MAX_RESPONSE_SIZE': '100000000',
            'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'
        }
    },    
)



763104351884.dkr.ecr.eu-west-1.amazonaws.com/pytorch-inference:1.11.0-gpu-py38
sagemaker-monai-brain-tumor-1654754484


In [11]:
print(model_name)
endpoint_config_name = f"monaiEndpointConfig-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1
        }
    ]
)
print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

sagemaker-monai-brain-tumor-1654754484
Created EndpointConfig: arn:aws:sagemaker:eu-west-1:617011600974:endpoint-config/monaiendpointconfig-2022-06-09-06-02-00


In [12]:
endpoint_name = f"sm-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)


In [13]:
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

Waiting for endpoint to create...
Endpoint Status: InService


In [15]:
import numpy as np
ep= endpoint_name
feed_data = "Test Data"
sm_runtime = boto3.Session().client('sagemaker-runtime')
r = sm_runtime.invoke_endpoint(EndpointName=ep, Body=feed_data)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "'collections.OrderedDict' object has no attribute 'to'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/sagemaker_inference/transformer.py", line 126, in transform
    result = self._transform_fn(self._model, input_data, content_type, accept)
  File "/opt/conda/lib/python3.8/site-packages/sagemaker_inference/transformer.py", line 216, in _default_transform_fn
    prediction = self._predict_fn(data, model)
  File "/opt/ml/model/code/inference.py", line 78, in predict_fn
    model = model.to(device)
AttributeError: 'collections.OrderedDict' object has no attribute 'to'
". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/sm-2022-06-09-06-02-16 in account 617011600974 for more information.